**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np
import sympy as sp
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from class_utils import error_histogram

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/p5q7gzupa2ndw55/sigmoid_regression_data.csv?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Strojové učenie na báze optimalizácie

Ako ďalší príklad si uvedieme veľmi jednoduchú aplikáciu optimalizácie v rámci strojového učenia. Naším cieľom bude vykonať regresiu: dostaneme vstupné a výstupné dáta a budeme sa snažiť nájsť funkciu, ktorá produkuje takú závislosť.

Takúto úlohu je ľahké previesť na optimalizačný problém. Budeme predpokladať, že máme k dispozícii parametrickú funkciu $f_\theta(\mathbf{x})$, ktorej charakter je daný vektorom parametrov $\theta$. Naším cieľom je nájsť také parametre, pri ktorých bude $f_\theta(\mathbf{x})$ na našej dátovej množine robiť čo najmenšie chyby.

Dajme tomu, že dátová množina má vzorky v tvare $(\mathbf{x_i}, \mathbf{y_i})$, kde $\mathbf{x_i}$ je vstup a $\mathbf{y}_i$ je požadovaný výstup pre vzorku $i$. Potom sa dá náš cieľ opísať ako nasledujúci optimalizačný problém:

\begin{equation}
\theta^* = \underset{\theta}{\arg\min} \sum_{(\mathbf{x}_i, \mathbf{y}_i)} \mathcal{L}(f_\theta(\mathbf{x}_i), \mathbf{y}_i)
\end{equation}t.j. chceme nájsť taký vektor parametrov $\theta^*$, ktorý bude minimalizovať rozdiel medzi skutočnými a požadovanými výstupmi na celej dátovej množine v zmysle nejakej chybovej funkcie: napríklad **kvadratickej chyby** .

### Dátová množina

Definíciu nášho regresného problému začneme načítaním dátovej množiny – ide o dáta zo zašumenej sigmoidnej krivky, ktoré budeme načítavať z CSV súboru. Keďže toto načítanie a predspracovanie sme už raz realizovali, nebudeme na tomto mieste celý postup opakovať a zdrojový kód nasledujúcej bunky je skrytý.



In [ ]:
#@title -- Data Loading and Preprocessing; X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df = pd.read_csv("data/sigmoid_regression_data.csv")

# we create a discretized version of the y column
# to allow for stratification
kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

# we split the dataset into train and test
df_train, df_test = train_test_split(df, stratify=y_stratify,
                                 test_size=0.3, random_state=4)

# we specify the inputs and the outputs
categorical_inputs = []
numeric_inputs = ['x']
output = 'y'

# we create the pipeline
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

# we fit and apply the pipeline on the train set
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs]).reshape(-1)
Y_train = df_train[output].values

# we apply the same pipeline to the test set,
# taking care to use transform and not fit_transform
X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs]).reshape(-1)
Y_test = df_test[output].values

# we plot the data for visual inspection
plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()
plt.savefig("output/regression_data.pdf", bbox_inches='tight', pad_inches=0)

---
### Úloha 1: Definícia regresnej funkcie

Závislosť, ktorú sme si vyššie vizualizovali, sa tvarom nápadne podobá na sigmoidnú (logistickú) kriku, ktorá je definovaná nasledovne:
\begin{equation}
\sigma(x) = \frac{1}{1 + e^{-x}}.
\end{equation}
Zdá sa však, že je trochu posunutá v smere x-ovej osi a možno nemá rovnakú strmosť. Zostavme preto regresný model tak, že vstup sigmoidnej funkcie prejde najprv jednoducho lineárnou transformáciou, ktorej parametre $a$ a $c$ sa naučíme z dát. Celý regresný model potom bude vyzerať nasledovne:
\begin{align}
u &= ax + c \
\sigma(u) &= \frac{1}{1 + e^{-u}}.
\end{align}

Alebo v rámci jednej funkcie:
\begin{equation}
\mathrm{f}(x, a, c) = \frac{1}{1 + e^{-ax - c}}
\end{equation}

**Pomocou balíčka `sympy` symbolicky definujte náš regresný model ako funkciu $f(x, a, c)$.** 

---


In [ ]:
symx, syma, symc = sp.symbols('x a c')



symf =      # ----




f = lambdify([symx, syma, symc], symf, "numpy")

symf

Regresnú funkciu `f(x, a, c)` si kvôli kontrole vizualizujeme:



In [ ]:
#@title -- Regression Function Visualization -- { display-mode: "form" }
xx = np.linspace(-5, 5, 100)
a = 1; c = 0
yy = [f(x, a, c) for x in xx]

plt.plot(xx, yy)
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')

plt.savefig("output/sigmoid.pdf", bbox_inches="tight", pad_inches=0)

### Účelová funkcia a jej gradient

Ako sme už povedali vyššie, naším cieľom je minimalizovať chybu na dátovej množine, t.j.

\begin{equation}
\theta^* = \underset{\theta}{\arg\min} \sum_{(\mathbf{x}_i, \mathbf{y}_i)} \mathcal{L}(f_\theta(\mathbf{x}_i), \mathbf{y}_i)
\end{equation}
Vonkajšej sume sa pri určovaní gradientu nemusíme venovať. Z linearity derivácie vyplýva, že 

\begin{equation}
\nabla \sum_{(\mathbf{x}_i, \mathbf{y}_i)} \mathcal{L}(f_\theta(\mathbf{x}_i), \mathbf{y}_i) = \sum_{(\mathbf{x}_i, \mathbf{y}_i)} \nabla \mathcal{L}(f_\theta(\mathbf{x}_i), \mathbf{y}_i)
\end{equation}stačí nám teda riešiť vnútornú časť sumy a potom gradienty pre jednotlivé vzorky z dátovej množiny sčítať dokopy.

Povedzme, že ako chybovú funkciu použijeme kvadratickú chybu, t.j.:

\begin{equation}
\mathcal{L}(f_\theta(\mathbf{x}_i), \mathbf{y}_i) = \left(
    f_\theta(\mathbf{x}_i) - \mathbf{y}_i
\right)^2.
\end{equation}
Zadefinujme si ju teraz symbolicky a určme gradient (len podľa parametrov $a$ a $c$ pretože tých optimálne hodnoty ideme hľadať):



In [ ]:
symy = sp.symbols('y')
symL = (symf - symy)**2

In [ ]:
L = lambdify((symx, symy, syma, symc), symL, "numpy")

sym_grad_L = sp.Matrix([symL]).jacobian([syma, symc])
grad_L_func = lambdify((symx, symy, syma, symc), sym_grad_L, "numpy")

sym_grad_L

Pri takejto definícii do funkcie `grad_L` dosadíme za všetky potrebné argumenty ($x$, $y$, $a$, $c$), ale navracia sa len vektor s dvoma prvkami: parciálnou deriváciou podľa $a$ a podľa $c$. Napr.:



In [ ]:
x = 0; y = 1; a = 1; c = 0
print("L:\t\t{}".format(L(x, y, a, c)))
print("grad_L:\t\t{}".format(grad_L_func(x, y, a, c)))

### Chyba a gradient pre všetky vzorky

Ako už vieme, celkovú chybu dostaneme ako súčet chýb pre jednotlivé vzorky a rovnako aj celkový gradient bude súčtom všetkých jednotlivých gradientov. Zadefinujme si teda funkcie, ktoré nám pomôžu oba výsledky vyrátať:



In [ ]:
def sumL(a, c, X, Y):
    L_sum = 0
    
    for x, y in zip(X, Y):
        L_sum += L(x, y, a, c)
        
    return L_sum

In [ ]:
def grad_sumL(a, c, X, Y):
    grad_sum = np.zeros(2)
    
    for x, y in zip(X, Y):
        grad_sum = grad_sum + grad_L_func(x, y, a, c)
        
    return grad_sum

Že všetko správne funguje môžeme otestovať na príklade: 



In [ ]:
print(sumL(a, c, X_train, Y_train))
print(grad_sumL(a, c, X_train, Y_train))

### Minimalizácia účelovej funkcie

Ďalej už len použijeme funkciu `minimize`, aby sme chybu minimalizovali a zobrazíme výslednú regresnú závislosť. Minimalizácia sa vykoná nasledovne:



In [ ]:
res = minimize(fun=lambda xx: sumL(*xx, X_train, Y_train),
               x0=np.random.uniform(0, 1, 2),
               method='L-BFGS-B',
               jac=lambda xx: grad_sumL(*xx, X_train, Y_train)
              )

a, c = res.x

### Testovanie regresného modelu

Určíme výstupy výsledného regresného modelu na tréningových dátach a vypočítame chybové ukazovatele.



In [ ]:
y_test = [f(x, a, c) for x in X_test]

In [ ]:
mse = mean_squared_error(Y_test, y_test)
mae = mean_absolute_error(Y_test, y_test)

print("MSE: {}".format(mse))
print("MAE: {}".format(mae))

Tak ako v jednom z predchádzajúcich notebook-ov, aj tu si pre úplnejšiu predstavu môžeme zobraziť aj histogram chýb:



In [ ]:
#@title -- Histogram of Outputs and Errors -- { display-mode: "form" }
plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)
plt.savefig("output/error_output_histogram.pdf", bbox_inches='tight', pad_inches=0)

Ešte užitočnejšie bude vizualizovať si výslednú regresnú funkciu a pôvodné dáta – vzhľadom na to, že dáta sú len 2-rozmerné, je to možné urobiť a poskytne nám to silnú intuíciu o tom, či sa regresná závislosť správa korektne:



In [ ]:
#@title -- Regression Curve vs. Data -- { display-mode: "form" }
x_min = min(np.min(X_train), np.min(X_test))
x_max = max(np.max(X_train), np.max(X_test))

xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))
yy = [f(x, a, c) for x in xx]

plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")

plt.plot(xx, yy, label="regression curve", c='k')

plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

plt.savefig("output/regression.pdf", bbox_inches="tight", pad_inches=0)

### Jednoduchšia implementácia regresie pomocou `curve_fit`

V praxi existujú samozrejme na realizáciu regresie aj jednoduchšie nástroje. Dobrým príkladom všeobecnej funkcie je `scipy.optimize.curve_fit`. Pri jej použítí stačí v predpísanej forme zadať regresnú funkciu (prípadne jej gradient) a nie je nutné zaoberať sa vecami ako je iterácia cez dátovú množinu a pod.

Vyššie uvedený príklad by sme pomocou `scipy.optimize.curve_fit` vedeli zjednodušiť takto:



In [ ]:
def sigmoid(x, a, c):
    return 1 / (1 + np.exp(-a*x - c))

In [ ]:
popt, _ = curve_fit(sigmoid, X_train.reshape(-1), Y_train.reshape(-1))

#### Vyhodnotenie na testovacích dátach



In [ ]:
y_test = sigmoid(X_test, *popt)

In [ ]:
mse = mean_squared_error(Y_test, y_test)
mae = mean_absolute_error(Y_test, y_test)

print("MSE: {}".format(mse))
print("MAE: {}".format(mae))

In [ ]:
#@title -- Regression Curve vs. Data -- { display-mode: "form" }
x_min = min(np.min(X_train), np.min(X_test))
x_max = max(np.max(X_train), np.max(X_test))

xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))
yy = sigmoid(xx, *popt)

plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")

plt.plot(xx, yy, label="regression curve")

plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

plt.savefig("output/regression2.pdf", bbox_inches="tight", pad_inches=0)